In [ ]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'

# send a GET request to the URL to download the file.
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response
# to it in chunks.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && git checkout 2b26469
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

In [25]:
!pip freeze | grep langchain
!pip freeze | grep gpt4all

langchain==0.0.208
langchainplus-sdk==0.0.20
gpt4all==0.2.3


In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

local_path = ("./models/ggml-model-q4_0.bin")
# local_path = ("./models/gpt4all-lora-quantized-ggml.bin")
callbacks = [StreamingStdOutCallbackHandler()]
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# # If you want to use a custom model add the backend parameter
# # Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.run(question)


ValidationError: 1 validation error for GPT4All
__root__
  No corresponding model for provided filename gpt4all-lora-quantized-ggml.bin.
            If this is a custom model, make sure to specify a valid model_type.
             (type=value_error)